In [1]:
'''
Answer in this format 'value unit' using the text on the image, What is the item weight?
'''

"\nAnswer in this format 'value unit' using the text on the image, What is the item weight?\n"

In [1]:
import pandas as pd
import numpy as np
import json as js

import sys
sys.path.append('./student_resource 3/src')

from utils import *
from constants import *
from pathlib import Path

In [2]:
data = pd.read_csv("student_resource 3/dataset/train.csv")

In [3]:
data.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [4]:
NUM_SAMPLES = 200

In [5]:
max_len = data.shape[0]
print(max_len)

263859


In [6]:
nums = np.random.randint(0, max_len-1, NUM_SAMPLES)

In [7]:
qdict = {}

In [8]:
with open("questions.jsonl", "w") as f:
    cnt = 0
    for i in nums:
        url, grp_id, e_name, e_val = data.iloc[i, :]
        f_name = Path(url).name
        download_image(url, "images")
        cnt += 1
        print(f"{cnt} q{i} : {e_name} : {e_val}")
        
        qdict[f"q{i}"] = (e_name, e_val)
        
        dct = {"question_id": f"q{i}", "text": f"Answer in this format 'value <space> unit' and unit shouldn't be in short form and use 2 decimal point precision when required, using the text on the image, What is the {' '.join(e_name.split('_'))}?", "image": f"{f_name}"}
        
        f.write(js.dumps(dct) + '\n')
        

1 q253953 : height : 360.0 millimetre
2 q70858 : item_weight : 500 gram
3 q74015 : item_weight : 330 gram
4 q109571 : item_weight : 3.29 kilogram
5 q87091 : voltage : 36.0 volt
6 q154876 : width : 5.4 centimetre
7 q206398 : depth : 15.5 centimetre
8 q147908 : width : 10.0 centimetre
9 q196179 : depth : 26.0 inch
10 q38883 : item_weight : 5.0 gram
11 q51642 : item_weight : 1.0 pound
12 q12169 : item_weight : 24.0 gram
13 q135506 : width : 80.0 centimetre
14 q189547 : depth : 40.0 centimetre
15 q261781 : height : 68.5 centimetre
16 q73528 : item_weight : 3.5 ounce
17 q63234 : item_weight : 20.0 ounce
18 q181406 : depth : 261.0 centimetre
19 q228969 : height : 18.5 centimetre
20 q201807 : height : 71.0 centimetre
21 q144530 : width : 50.5 inch
22 q262232 : height : 27.0 centimetre
23 q229879 : height : 19.0 centimetre
24 q26202 : item_weight : 47.6 gram
25 q162061 : width : 3.34 inch
26 q172991 : height : 6.5 centimetre
27 q10983 : voltage : 21.6 volt
28 q184385 : depth : 7.6 inch
29 q957

In [9]:
import re

In [10]:
entity_unit_available = {
    "centimetre", "foot", "millimetre", "metre", "inch", "yard",
    "milligram", "kilogram", "microgram", "gram", "ounce", "ton", "pound",
    "millivolt", "kilovolt", "volt", "kilowatt", "watt",
    "cubic foot", "microlitre", "cup", "fluid ounce", "centilitre",
    "imperial gallon", "pint", "decilitre", "litre", "millilitre", "quart",
    "cubic inch", "gallon"
}

regexed_mapping = {
    "mg": "milligram",
    "g": "gram",
    "kg": "kilogram",
    "cm": "centimetre",
    "mm": "millimetre",
    "m": "metre",
    "ml": "millilitre",
    "l": "litre",
    "mv": "millivolt",
    "kv": "kilovolt",
    "v": "volt",
    "kw": "kilowatt",
    "w": "watt",
    "oz": "ounce",
    "lb": "pound",
}
patterns = {
    'item_weight': r'(\d+(\.\d+)?\s?(mg|milligram|g|gram|kg|kilogram|microgram|ounce|ton|pound))',
    'height': r'(\d+(\.\d+)?\s?(mm|millimeter|cm|centimeter|m|meter|inch|foot|yard))',
    'depth': r'(\d+(\.\d+)?\s?(mm|millimeter|cm|centimeter|m|meter|inch|foot|yard))',
    'item_volume': r'(\d+(\.\d+)?\s?(ml|milliliter|l|liter|cubic foot|microliter|cup|fluid ounce|centiliter|imperial gallon|pint|deciliter|quart|cubic inch|gallon))',
    'maximum_weight_recommendation': r'(\d+(\.\d+)?\s?(mg|milligram|kg|kilogram|g|gram|microgram|ounce|ton|pound))',
    'voltage': r'(\d+(\.\d+)?\s?(mV|millivolt|kV|kilovolt|V|volt))',
    'wattage': r'(\d+(\.\d+)?\s?(kW|kilowatt|W|watt))'
}


def convert_to_grams(value, unit):
    conversions = {
        'mg': 0.001,
        'g': 1,
        'kg': 1000,
        'gram': 1,
        'kilogram': 1000
    }
    return float(value) * conversions.get(unit.lower(), 0)

def convert_to_meters(value, unit):
    conversions = {
        'mm': 0.001,
        'cm': 0.01,
        'm': 1,
        'millimeter': 0.001,
        'centimeter': 0.01,
        'meter': 1
    }
    return float(value) * conversions.get(unit.lower(), 0)

def convert_to_liters(value, unit):
    conversions = {
        'ml': 0.001,
        'l': 1,
        'litre': 1,
        'millilitre': 0.001
    }
    return float(value) * conversions.get(unit.lower(), 0)

def convert_to_base_voltage(value, unit):
    # Voltage is already in volts, no need for conversion
    return float(value)

def convert_to_base_wattage(value, unit):
    # Wattage is already in watts, no need for conversion
    return float(value)

def regex_based_extraction_maxa(text, attribute):
    pattern = patterns.get(attribute.lower(), None)
        
    if not pattern:
        return None

    matches = re.findall(pattern, text, re.IGNORECASE)
    if not matches:
        return None

    max_value = 0
    max_match = None

    for _match in matches:
        if len(_match[0].split(' '))==1:
            # seperate int
            _int=''
            _unit=''
            for i in range(len(_match[0])):
                if _match[0][i].isdigit():
                    _int+=_match[0][i]
                else:
                    break
            _unit=_match[0][len(_int):]
            value, unit = _int, _unit
        elif len(_match[0].split(' '))!=2:
            continue
        else:
            value, unit = _match[0].split(' ')



        unit=unit.lower()
        if unit in regexed_mapping:
            unit=regexed_mapping[unit]
        max_match = value + ' ' + unit
        return max_match
        # Convert based on the attribute type
        if attribute.lower() in ['item_weight', 'maximum_weight_recommendation']:
            converted_value = convert_to_grams(value, unit)
        elif attribute.lower() in ['height', 'depth']:
            converted_value = convert_to_meters(value, unit)
        elif attribute.lower() == 'item_volume':
            converted_value = convert_to_liters(value, unit)
        elif attribute.lower() == 'voltage':
            converted_value = convert_to_base_voltage(value, unit)
        elif attribute.lower() == 'wattage':
            converted_value = convert_to_base_wattage(value, unit)
        else:
            continue  # Skip unknown attributes

        # Check if this is the maximum value found
        if converted_value > max_value:
            max_value = converted_value
            unit=unit.lower()
            if unit in regexed_mapping:
                unit=regexed_mapping[unit]
            max_match = value + ' ' + unit
    return max_match  # Return the match with the highest unit

In [27]:
print("Question_id : entity_name : entity_value : predicted_value")
with open("answers.jsonl", "r") as f:
    for line in f:
        dat = js.loads(line)
        
        en_name, en_val = qdict[dat['question_id']]
        
        print(f"{dat['question_id']} : {en_name} : {en_val} : {regex_based_extraction_maxa(dat['text'], en_name)}")

Question_id : entity_name : entity_value : predicted_value


NameError: name 'qdict' is not defined